In [ ]:
import os
!git clone https://github.com/Maria-Elisa-M/BW_prediction_keypoint.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# uncomment to install
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.8/921.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
os.chdir('BW_prediction_keypoint')
main_dir = 'depth'
# Define paths for images and YOLO model
image_dir = os.path.join(main_dir, 'Images')
output_mask_dir = os.path.join(main_dir, 'output', 'masks')
output_vis_dir = os.path.join(main_dir, 'output', 'masks_vis')
model_path = os.path.join(main_dir, 'segment/train/stream3/train/weights/best.pt')

In [6]:
# Create output directories if they don't exist
os.makedirs(output_mask_dir, exist_ok=True)
os.makedirs(output_vis_dir, exist_ok=True)

# Load YOLO model
model = YOLO(model_path)

# Process each image in the directory
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(image_dir, filename)
        mask_output_path = os.path.join(output_mask_dir, filename)
        vis_output_path = os.path.join(output_vis_dir, filename)

        # Load the image
        image = cv2.imread(image_path)

        # Run inference
        results = model(image_path)[0]  # Get first result

        # Create an empty mask
        mask = np.zeros((results.orig_img.shape[0], results.orig_img.shape[1]), dtype=np.uint8)

        # Extract masks if available
        if results.masks is not None:
            largest_area = 0
            largest_contour = None
            for seg in results.masks.xy:
                pts = np.array(seg, dtype=np.int32)
                area = cv2.contourArea(pts)
                if area > largest_area:
                    largest_area = area
                    largest_contour = pts

            # Fill only the largest object
            if largest_contour is not None:
                cv2.fillPoly(mask, [largest_contour], 255)  # Fill the mask with white (255)

        # Save the black and white mask
        cv2.imwrite(mask_output_path, mask)

        # Overlay the mask on the original image
        overlay = cv2.addWeighted(image, 0.6, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), 0.4, 0)

        # Save the overlayed image
        cv2.imwrite(vis_output_path, overlay)
        print(f"Saved mask: {mask_output_path}")
        print(f"Saved mask overlay: {vis_output_path}")

print("Processing complete.")



image 1/1 /content/drive/MyDrive/guilherme/depth/Images/4505_20220808_072116_depth_frame_0.png: 384x672 1 cow, 482.6ms
Speed: 22.8ms preprocess, 482.6ms inference, 52.5ms postprocess per image at shape (1, 3, 384, 672)
Saved mask: /content/drive/MyDrive/guilherme/depth/output/masks/4505_20220808_072116_depth_frame_0.png
Saved mask overlay: /content/drive/MyDrive/guilherme/depth/output/masks_vis/4505_20220808_072116_depth_frame_0.png

image 1/1 /content/drive/MyDrive/guilherme/depth/Images/4503_20220726_093253_depth_frame_0.png: 384x672 1 cow, 207.6ms
Speed: 3.0ms preprocess, 207.6ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 672)
Saved mask: /content/drive/MyDrive/guilherme/depth/output/masks/4503_20220726_093253_depth_frame_0.png
Saved mask overlay: /content/drive/MyDrive/guilherme/depth/output/masks_vis/4503_20220726_093253_depth_frame_0.png

image 1/1 /content/drive/MyDrive/guilherme/depth/Images/4498_20220808_080724_depth_frame_450.png: 384x672 1 cow, 477.5ms
Spee